In [1]:
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
from technicals.indicators import *
from technicals.patterns import apply_patterns
from technicals.patternsInRange import apply_patterns_in_range

from guruTester import GuruTester,GuruTester2
from plotting import CandlePlot
import MetaTrader5 as mt5
from datetime import datetime
from getCandle import Rates
from sklearn.neural_network import MLPClassifier
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler  
import pickle

In [2]:
pair = "GBPUSD"
NB_H1_CANDLES = 20000

In [3]:
rates = Rates(pair, NB_H1_CANDLES, mt5.TIMEFRAME_H1)
SPREAD = rates.get_spread()
df_an = rates.get_rates_from_now()
df_an.drop(NB_H1_CANDLES-1,inplace=True) 
df_an.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)
SPREAD

0.00010000000000021103

In [4]:
SPREAD = (SPREAD//1e-5)*1e-5 # todo: do it right!!!
SPREAD # 0

0.0001

In [5]:
def featureExtraction(df):
    candle_patterns = ['HANGING_MAN', 'SHOOTING_STAR',
           'SPINNING_TOP', 'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP',
           'TWEEZER_BOTTOM', 'MORNING_STAR', 'EVENING_STAR']
    df = apply_patterns_in_range(df)
    for cp in candle_patterns:
        df[cp] = df[cp].astype(float)

    df = BollingerBandsNonLinear(df)
#     df = ATRFeature(df)
    df = KeltnerChannelsNonLinear(df)
    df = RSINonLinear(df)
    df = MACDNonLinear(df)
    df = BREAKOUT(df)
    df = ichimokuNonLinear(df)
#     df['pivots_l'] = False
#     df['pivots_h'] = False

#     df['low_perv'] = df.low.shift(1)
#     df['high_perv'] = df.high.shift(1)
#     df['low_next'] = df.low.shift(-1)
#     df['high_next'] = df.high.shift(-1)
#     df['pivots_l'] = (df.low_perv > df.low) & (df.low_next > df.low)
#     df['pivots_h'] = (df.high_perv < df.high) & (df.high_next < df.high)
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

#     df.pivots_l = df.pivots_l.astype(float)
#     df.pivots_h = df.pivots_h.astype(float)
    return df

In [6]:
df_an = featureExtraction(df_an)

C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:215: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_pivot_h'].iloc[i] = last_pivot_h
C:\Users\meysam\Desktop\algotramic\technicals\indicators.py:216: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['last_pivot_l'].iloc[i] = last_pivot_l


In [7]:
BUY = 1
SELL = -1
NONE = 0

df_an['SIGNAL'] = 0
df_an['TP'] = 0
df_an['SL'] = 0
for i in range(1,len(df_an)):
    if df_an.pivots_l[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.low[j] <= df_an.low[i-1] ):
                break
            else:
                if(df_an.high[j] > 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = BUY
                    break
        
        df_an['SL'][i] = df_an.low[i-1] 
        df_an['TP'][i] = 2*(df_an.close[i] - df_an.low[i-1] ) + df_an.close[i]
                    
    if df_an.pivots_h[i-1]:
        for j in range(i,len(df_an)):
            if( df_an.high[j] >= df_an.high[i-1] ):
                break
            else:
                if(df_an.low[j] < 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]):
                    df_an.SIGNAL[i] = SELL
                    break
                    
        df_an['SL'][i] = df_an.high[i-1] 
        df_an['TP'][i] = 2*(df_an.close[i] - df_an.high[i-1] ) + df_an.close[i]
                    

C:\Users\meysam\AppData\Local\Temp\ipykernel_13004\3582309630.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an.SIGNAL[i] = BUY
C:\Users\meysam\AppData\Local\Temp\ipykernel_13004\3582309630.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['SL'][i] = df_an.low[i-1]
C:\Users\meysam\AppData\Local\Temp\ipykernel_13004\3582309630.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_an['TP'][i] = 2*(df_an.close[i] - df_

In [8]:
df_an.columns

Index(['time', 'open', 'high', 'low', 'close', 'full_range', 'body_lower',
       'body_upper', 'body_bottom_perc', 'body_top_perc', 'body_perc',
       'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'BB_MA_cross', 'BB_UP_touch',
       'BB_LW_touch', 'EMA_cross', 'KeUp_touch', 'KeLo_touch', 'RSI_level',
       'MACDHIST_level', 'pivots_l', 'pivots_h', 'low_perv', 'high_perv',
       'low_next', 'high_next', 'last_pivot_h', 'last_pivot_l', 'close_perv',
       'BREAK_UP', 'BREAK_DOWN', 'Tenkan_sen', 'Kijun_sen', 'Senkou_Span_A',
       'Senkou_Span_B', 'Tenkan_sen_touch', 'Kijun_sen_touch',
       'Senkou_Span_A_touch', 'Senkou_Span_B_touch', 'SIGNAL', 'TP', 'SL']

In [9]:

feature_col = [
    'BB_MA_cross', 'BB_UP_touch','BB_LW_touch',
    'EMA_cross', 'KeUp_touch', 'KeLo_touch', 
    'RSI_level','MACDHIST_level',
#      'BREAK_UP', 'BREAK_DOWN',
        'Tenkan_sen_touch', 'Kijun_sen_touch',
       'Senkou_Span_A_touch', 'Senkou_Span_B_touch',
#         'full_range',# this is added ...
#        'body_lower', 'body_upper', 
#         'body_bottom_perc', 'body_top_perc',
#        'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
#     'mid_point', 'mid_point_prev_2', 'body_size_prev',
#        'direction_prev', 'direction_prev_2', 'body_perc_prev','body_perc_prev_2',
        'direction',
        'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING',
    'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR'
]

In [10]:
X = []
Y = [] 
indeces = []
for i in range(4,len(df_an)):
    if df_an.pivots_h[i-1] or df_an.pivots_l[i-1] :
        feature = []
        feature.append(df_an.pivots_h[i-1])
        feature.append(df_an.pivots_l[i-1])
        feature.append(df_an.pivots_h[i-2])
        feature.append(df_an.pivots_l[i-2])
        feature.append(df_an.pivots_h[i-3])
        feature.append(df_an.pivots_l[i-3])
        feature.append(df_an.pivots_h[i-4])
        feature.append(df_an.pivots_l[i-4])
        for fc in feature_col:
            feature.append(df_an[fc][i])
            feature.append(df_an[fc][i-1])
            feature.append(df_an[fc][i-2])
            feature.append(df_an[fc][i-3])
            feature.append(df_an[fc][i-4])
#             feature.append(df_an[fc][i-5])
#             feature.append(df_an[fc][i-6])
#             feature.append(df_an[fc][i-8])
#             feature.append(df_an[fc][i-9])
#             feature.append(df_an[fc][i-10])

        X.append(feature)
        Y.append(df_an.SIGNAL[i])
        indeces.append(i)
        

In [11]:
SPLIT_INDEX = int(0.8*len(X))
X_train = X[:SPLIT_INDEX]
Y_train = Y[:SPLIT_INDEX]
indeces_train = indeces[:SPLIT_INDEX]
X_test = X[SPLIT_INDEX:]
Y_test = Y[SPLIT_INDEX:]
indeces_test = indeces[SPLIT_INDEX:]

In [12]:
len(feature)

118

In [35]:
clf = MLPClassifier( solver='adam', alpha=1e-3,learning_rate_init=1e-4, # adam
                    hidden_layer_sizes=(200,200), 
                    random_state=1,
                    max_iter=2000,
                    verbose=True,warm_start = True,
                    n_iter_no_change=40,
#                     early_stopping = True,
#                     validation_fraction = 0.2 # default is 0.1
                   )

In [36]:
clf.fit(X_train, Y_train)

Iteration 1, loss = 1.02563261
Iteration 2, loss = 0.93098099
Iteration 3, loss = 0.89479090
Iteration 4, loss = 0.86327695
Iteration 5, loss = 0.83509599
Iteration 6, loss = 0.81012725
Iteration 7, loss = 0.78911849
Iteration 8, loss = 0.77103731
Iteration 9, loss = 0.75617944
Iteration 10, loss = 0.74355845
Iteration 11, loss = 0.73220506
Iteration 12, loss = 0.72309546
Iteration 13, loss = 0.71502684
Iteration 14, loss = 0.70723290
Iteration 15, loss = 0.70055997
Iteration 16, loss = 0.69476786
Iteration 17, loss = 0.68942703
Iteration 18, loss = 0.68429432
Iteration 19, loss = 0.68033769
Iteration 20, loss = 0.67582066
Iteration 21, loss = 0.67133498
Iteration 22, loss = 0.66780415
Iteration 23, loss = 0.66458392
Iteration 24, loss = 0.66062715
Iteration 25, loss = 0.65639779
Iteration 26, loss = 0.65361232
Iteration 27, loss = 0.65023482
Iteration 28, loss = 0.64668371
Iteration 29, loss = 0.64467046
Iteration 30, loss = 0.64084085
Iteration 31, loss = 0.63746254
Iteration 32, los

Iteration 253, loss = 0.18255709
Iteration 254, loss = 0.18004616
Iteration 255, loss = 0.17887277
Iteration 256, loss = 0.17671466
Iteration 257, loss = 0.17796461
Iteration 258, loss = 0.17608684
Iteration 259, loss = 0.17378904
Iteration 260, loss = 0.17374260
Iteration 261, loss = 0.17204091
Iteration 262, loss = 0.17103557
Iteration 263, loss = 0.16835464
Iteration 264, loss = 0.16811755
Iteration 265, loss = 0.16673038
Iteration 266, loss = 0.16632349
Iteration 267, loss = 0.16604413
Iteration 268, loss = 0.16395305
Iteration 269, loss = 0.16141031
Iteration 270, loss = 0.15907865
Iteration 271, loss = 0.15829571
Iteration 272, loss = 0.15775829
Iteration 273, loss = 0.15650871
Iteration 274, loss = 0.15519404
Iteration 275, loss = 0.15257791
Iteration 276, loss = 0.15153756
Iteration 277, loss = 0.15128415
Iteration 278, loss = 0.15044233
Iteration 279, loss = 0.14952291
Iteration 280, loss = 0.14931449
Iteration 281, loss = 0.14657701
Iteration 282, loss = 0.14611580
Iteration 

Iteration 502, loss = 0.01694847
Iteration 503, loss = 0.01693053
Iteration 504, loss = 0.01748460
Iteration 505, loss = 0.01700173
Iteration 506, loss = 0.01741126
Iteration 507, loss = 0.01621666
Iteration 508, loss = 0.01584131
Iteration 509, loss = 0.01594591
Iteration 510, loss = 0.01548173
Iteration 511, loss = 0.01510713
Iteration 512, loss = 0.01538808
Iteration 513, loss = 0.01507013
Iteration 514, loss = 0.01464687
Iteration 515, loss = 0.01507781
Iteration 516, loss = 0.01449074
Iteration 517, loss = 0.01430136
Iteration 518, loss = 0.01411164
Iteration 519, loss = 0.01432525
Iteration 520, loss = 0.01375639
Iteration 521, loss = 0.01374047
Iteration 522, loss = 0.01442339
Iteration 523, loss = 0.01395868
Iteration 524, loss = 0.01315931
Iteration 525, loss = 0.01340783
Iteration 526, loss = 0.01327864
Iteration 527, loss = 0.01271041
Iteration 528, loss = 0.01258960
Iteration 529, loss = 0.01264955
Iteration 530, loss = 0.01237426
Iteration 531, loss = 0.01238417
Iteration 

Iteration 751, loss = 0.00204931
Iteration 752, loss = 0.00191263
Iteration 753, loss = 0.00154138
Iteration 754, loss = 0.00217507
Iteration 755, loss = 0.00190233
Iteration 756, loss = 0.00210972
Iteration 757, loss = 0.00161916
Iteration 758, loss = 0.00197744
Iteration 759, loss = 0.00162759
Iteration 760, loss = 0.00166180
Iteration 761, loss = 0.00179284
Iteration 762, loss = 0.00194301
Iteration 763, loss = 0.00193898
Iteration 764, loss = 0.00188550
Iteration 765, loss = 0.00182553
Iteration 766, loss = 0.00201929
Iteration 767, loss = 0.00209381
Iteration 768, loss = 0.00153628
Iteration 769, loss = 0.00138279
Iteration 770, loss = 0.00173650
Iteration 771, loss = 0.00191900
Iteration 772, loss = 0.00177003
Iteration 773, loss = 0.00160692
Iteration 774, loss = 0.00145855
Iteration 775, loss = 0.00140085
Iteration 776, loss = 0.00160498
Iteration 777, loss = 0.00136898
Iteration 778, loss = 0.00178256
Iteration 779, loss = 0.00206271
Iteration 780, loss = 0.00271420
Iteration 

MLPClassifier(hidden_layer_sizes=(200, 200), learning_rate_init=0.0001,
              max_iter=2000, n_iter_no_change=40, random_state=1, verbose=True,
              warm_start=True)

In [37]:
y_perdict = clf.predict(X_test)

In [38]:
clf.score(X_train, Y_train)

0.9998550934647152

In [39]:
clf.score(X_test, Y_test)

0.555619930475087

In [40]:
conf = confusion_matrix(Y_test, y_perdict)
conf

array([[149, 196,   7],
       [191, 667, 178],
       [ 11, 184, 143]], dtype=int64)

In [41]:
df_test = df_an[['time', 'open', 'high', 'low', 'close','TP','SL','pivots_l','pivots_h']].iloc[indeces_test[0]:].copy()
df_test['SIGNAL'] = 0
for i,idx in enumerate(indeces_test):
    if i > 0:
#         df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_l[idx-1]==1 and y_perdict[i] ==BUY:
            df_test.SIGNAL.loc[idx] = y_perdict[i]
        if df_test.pivots_h[idx-1]==1 and y_perdict[i] ==SELL:
            df_test.SIGNAL.loc[idx] = y_perdict[i]

df_test.reset_index(drop=True, inplace=True)

C:\Users\meysam\AppData\Local\Temp\ipykernel_13004\3314508941.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\meysam\AppData\Local\Temp\ipykernel_13004\3314508941.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
rates = Rates(pair, 13*len(df_test), mt5.TIMEFRAME_M5)
df_m5 = rates.get_rates_from_now()
df_m5.reset_index(drop=True, inplace=True)
df_m5.drop(['tick_volume', 'spread', 'real_volume'], axis=1, inplace=True)

In [43]:
gt = GuruTester2(
        df_test,
        df_m5,
        SPREAD,
        use_spread=True
    )
df_res_m5 = gt.run_test()

prepare_data...
run_test...
Result:
win rate:  0.3803680981595092
-1.0    404
 2.0    248
Name: result, dtype: int64


In [44]:
df_res_m5['balance'] = 100
loss_margin = 0.03
perv_balance = 100

# for index, row in df_res_m5.iterrows():
for i in range(len(df_res_m5)):
    if(df_res_m5.result[i] == -1):
        loss_ratio = 1 # abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(loss_ratio)
        df_res_m5['balance'][i] = perv_balance*(1-loss_ratio*loss_margin) 
    elif(df_res_m5.result[i] == 2):
        profit_to_loss = abs( (df_res_m5['start_price'][i]-df_res_m5['trigger_price'][i])/((df_res_m5['start_price'][i]-df_res_m5['SL'][i])) )
#         print(profit_to_loss)
        df_res_m5['balance'][i] = perv_balance*(1+profit_to_loss*loss_margin) 
    perv_balance = df_res_m5['balance'][i]

C:\Users\meysam\AppData\Local\Temp\ipykernel_13004\3900743223.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
df_res_m5_plot = df_res_m5#.loc[:100]
df_res_m5_plot['time'] = df_res_m5_plot['end_time']
cp = CandlePlot(df_res_m5_plot, candles=False)
print("min balance ",min(df_res_m5['balance']))
cp.show_plot(line_traces=['balance'])

min balance  62.65680962733082
